**1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.**

**2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by "-".**

In [1]:
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import requests
import re
import pandas as pd
import time
from selenium.webdriver.common.keys import Keys

import warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome(r'C:\Users\Sahil\Downloads\chromedriver.exe')

In [7]:
url = "https://www.amazon.in/"
driver.get(url)

In [8]:
#locating the search bar
search_bar=driver.find_element_by_xpath("/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input")
search_bar.send_keys('toys')

time.sleep(2)
#locating the button and clicking it toh search for toys
button=driver.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input')
button.click()

In [18]:
produrl =[]

produrl_tag = driver.find_elements_by_xpath("//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")

for u in produrl_tag:
    try: 
        produrl.append(u.get_attribute("href"))
    except:
        produrl.append('-')

In [20]:
i=2

while(i>0): 
    nextbtn = driver.find_element_by_xpath("//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']")
    nextbtn.click()
    time.sleep(3)
    produrl_tag = driver.find_elements_by_xpath("//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
    for u in produrl_tag:
          produrl.append(u.get_attribute("href"))
    i=i-1

In [24]:
#creating the empty list
brandname=[]
productname=[]
price=[]
ret_exch=[]
exp_del=[]
avail=[]

In [25]:
for i in produrl:
    driver.get(i)
    time.sleep(5)
    try:
        brandtag = driver.find_element_by_xpath("//*[@id='productOverview_feature_div']/div/table/tbody/tr[2]/td[2]/span")
        brandname.append(brandtag.text)
    except:
        brandname.append('-')
    
    try:
        prodtag = driver.find_element_by_xpath("//*[@id='productTitle']")
        productname.append(prodtag.text)
    except:
        productname.append('-')
        
    try:
        pricetag = driver.find_element_by_xpath("//*[@id='corePriceDisplay_desktop_feature_div']/div[1]/span[2]/span[2]/span[2]")
        price.append(pricetag.text)
    except:
        price.append('-')
        
    try:
        retextag = driver.find_element_by_xpath("//*[@id='RETURNS_POLICY']/span/div[2]/a")
        ret_exch.append(retextag.text.replace('\n',' ').replace('   ',''))
    except:
        ret_exch.append('-')
        
    try:
        expdeltag = driver.find_element_by_xpath("//*[@id='mir-layout-DELIVERY_BLOCK-slot-DELIVERY_MESSAGE']/b")
        exp_del.append(expdeltag.text)
    except:
        exp_del.append('-')
        
    try:
        availtag = driver.find_element_by_xpath("//*[@id='availability']")
        avail.append(availtag.text)
    except:
        avail.append('-')

In [26]:
toys = {'Brand Name':brandname,
        'Product Name':productname,
        'Price(₹)':price,
        'Return/Exchange':ret_exch,
        'Expected Delivery':exp_del,
        'Availablitiy':avail,
        'Product URL':produrl}

toys  = pd.DataFrame(toys)

In [27]:
toys

,Brand Name,Product Name,Price(₹),Return/Exchange,Expected Delivery,Availablitiy,Product URL
0,Wembley,Wembley Dancing Cactus Toy for Baby Funny Cact...,-,7 Days Returnable,"Saturday, March 5",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
1,Kid,"HILLSTAR Dancing Cactus Talking Toy, Cactus Pl...",599,7 Days Returnable,"Saturday, March 5",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
2,Genius Box - Play some Learning,Genius Box - Play Some Learning Educational To...,"1,312",7 Days Returnable,"Thursday, March 3",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
3,Animals,West Paw Design Zogoflex® Toppl Interactive Tr...,"5,500",7 Days Returnable,March 21 - 22,Usually dispatched in 4 to 5 days.,https://www.amazon.in/gp/slredirect/picassoRed...
4,SUPER TOY,SUPER TOY LCD Writing Tablet 8.5Inch E-Note Pad,299,7 Days Returnable,"Thursday, March 3",In stock.,https://www.amazon.in/SUPER-TOY-Writing-Tablet...
...,...,...,...,...,...,...,...
246,Medium,"BINGE Pop It Fidget Toys,Push Pop Bubble Fidge...",79,7 Days Returnable,"Sunday, March 6",In stock.,https://www.amazon.in/BINGE-Sensory-Special-Si...
247,-,Jungle Magic Doodle Waterz - Reusable I Water ...,-,7 Days Returnable,"Thursday, March 3",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
248,PLUSPOINT,pluspoint musical happy bus toys with xylophon...,899,7 Days Returnable,"Sunday, March 6",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
249,Play Figure,"Scross Doraemon (20 Cm, Blue)",581,7 Days Returnable,"Saturday, March 12",Usually dispatched in 4 to 5 days.,https://www.amazon.in/gp/slredirect/picassoRed...


In [28]:
toys.to_csv('Amazon_WebScraping_Q1and2.csv')

**3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.**

In [2]:
driver = webdriver.Chrome(r'C:\Users\Sahil\Downloads\chromedriver.exe')

In [3]:
url = "https://www.images.google.com/"
driver.get(url)

In [4]:
from selenium.webdriver.common.keys import Keys
srch = driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
srch.send_keys('fruits')
srch.send_keys(Keys.ENTER)

In [5]:
for _ in range(0,500):    
    driver.execute_script("window.scrollBy(0,10000)")

In [6]:
img = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
url = []
for i in img:
    sc = i.get_attribute('src')
    if sc is not None:
        if(sc[0:4] =='http'):
            url.append(sc)     

In [9]:
import requests
for i in range(len(url)):
    if i>=10:
        break
    print("Downloading {0} out of {1} images".format(i,10))
    resp = requests.get(url[i])
    file = open(r'C:\Users\Sahil\Documents\Flip Robo\Images from Q3\Fruits\fruit'+str(i)+'.jpg','wb')
    file.write(resp.content)

In [7]:
url = "https://www.images.google.com/"
driver.get(url)

In [8]:
srch = driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
srch.send_keys('cars')
srch.send_keys(Keys.ENTER)

In [9]:
for _ in range(0,500):    
    driver.execute_script("window.scrollBy(0,10000)")

In [10]:
car_img = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
car = []
for i in car_img:
    x = i.get_attribute('src')
    if x is not None:
        if(x[0:4]=='http'):
            car.append(x)

for i in range(len(car)):
    if i>=10:
        break
    respo = requests.get(car[i])
    print("downloading images {0} of {1}".format(i,10))
    file = open(r'C:\Users\Sahil\Documents\Flip Robo\Images from Q3\Cars\car'+str(i)+'.jpg','wb')
    file.write(respo.content)

downloading images 0 of 10
downloading images 1 of 10
downloading images 2 of 10
downloading images 3 of 10
downloading images 4 of 10
downloading images 5 of 10
downloading images 6 of 10
downloading images 7 of 10
downloading images 8 of 10
downloading images 9 of 10


In [15]:
url = "https://www.images.google.com/"
driver.get(url)

In [16]:
srch = driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
srch.send_keys('machine learning')
srch.send_keys(Keys.ENTER)

In [17]:
for i in range(0,500):
    driver.execute_script('window.scrollBy(0,10000)')

In [18]:
ml_img = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
ml = []

for i in ml_img:
    sc = i.get_attribute('src')
    if sc is not None:
        if(sc[0:4]=='http'):
            ml.append(sc)

for i in range(len(ml)):
    if i>=10:
        break
    print('downloading images {0} of {1}'.format(i,10))
    resp = requests.get(ml[i])
    file = open(r'C:\Users\Sahil\Documents\Flip Robo\Images from Q3\Machine Learning\ml'+str(i)+'.jpg',"wb")
    file.write(resp.content)

downloading images 0 of 10
downloading images 1 of 10
downloading images 2 of 10
downloading images 3 of 10
downloading images 4 of 10
downloading images 5 of 10
downloading images 6 of 10
downloading images 7 of 10
downloading images 8 of 10
downloading images 9 of 10


In [23]:
url = "https://www.images.google.com/"
driver.get(url)

In [24]:
srch = driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
srch.send_keys('guitar')
srch.send_keys(Keys.ENTER)

In [25]:
for i in range(0,500):
    driver.execute_script('window.scrollBy(0,10000)')

In [26]:
guitar_img = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
guitar = []

for i in guitar_img:
    sc = i.get_attribute('src')
    if sc is not None:
        if(sc[0:4]=='http'):
            guitar.append(sc)

for i in range(len(guitar)):
    if i>=10:
        break
    print('downloading images {0} of {1}'.format(i,10))
    resp = requests.get(guitar[i])
    file = open(r'C:\Users\Sahil\Documents\Flip Robo\Images from Q3\Guitar\guitar'+str(i)+'.jpg',"wb")
    file.write(resp.content)

downloading images 0 of 10
downloading images 1 of 10
downloading images 2 of 10
downloading images 3 of 10
downloading images 4 of 10
downloading images 5 of 10
downloading images 6 of 10
downloading images 7 of 10
downloading images 8 of 10
downloading images 9 of 10


In [27]:
url = "https://www.images.google.com/"
driver.get(url)

In [28]:
srch = driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
srch.send_keys('cakes')
srch.send_keys(Keys.ENTER)

In [29]:
for i in range(0,500):
    driver.execute_script('window.scrollBy(0,10000)')

In [30]:
cakes_img = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
cakes = []

for i in cakes_img:
    sc = i.get_attribute('src')
    if sc is not None:
        if(sc[0:4]=='http'):
            cakes.append(sc)

for i in range(len(cakes)):
    if i>=10:
        break
    print('downloading images {0} of {1}'.format(i,10))
    resp = requests.get(cakes[i])
    file = open(r'C:\Users\Sahil\Documents\Flip Robo\Images from Q3\Cakes\cakes'+str(i)+'.jpg',"wb")
    file.write(resp.content)

downloading images 0 of 10
downloading images 1 of 10
downloading images 2 of 10
downloading images 3 of 10
downloading images 4 of 10
downloading images 5 of 10
downloading images 6 of 10
downloading images 7 of 10
downloading images 8 of 10
downloading images 9 of 10


**4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV**

In [3]:
url = "https://www.flipkart.com/"
driver.get(url)

In [5]:
item = input("Enter a smartphone name to search:")

Enter a smartphone name to search:iphone


In [6]:
search = driver.find_element_by_xpath('/html/body/div/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')
search.send_keys(item)
searchbtn = driver.find_element_by_class_name('L0Z3Pu')
searchbtn.click()

In [7]:
produrl = []

produrl_tag = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
for u in produrl_tag:
     produrl.append(u.get_attribute("href"))

sep = ' | '

In [8]:
Brand = []
prod = []
Colour = []
RAM = []
Storage = []
PrimCamera = []
SecCamera = []
DispSize = []
BatteryCap = []
price = []

In [11]:
for i in produrl:
    driver.get(i)
    time.sleep(3)
    morebtn = driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']")
    morebtn.click()
    time.sleep(5)

#brand
    try:
        brandtag = driver.find_element_by_xpath("/html/body/div[1]/div/div[3]/div[1]/div[2]/div[1]/div[1]/div/div[4]/a")
        Brand.append(brandtag.text)
    except:
        Brand.append('-')
        
#product, i.e. smartphone name        
    try:
        prodtag = driver.find_element_by_xpath("//*[@id='container']/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span")
        prod.append(prodtag.text)
    except:
        prod.append('-')

#Color        
    try:
        Colourtag = driver.find_element_by_xpath("//*[@id='container']/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[2]/table/tbody/tr[4]/td[2]/ul/li")
        Colour.append(Colourtag.text)
    except NoSuchElementException:
        Colour.append('-')
        
#RAM
    try:
        RAMtag = driver.find_element_by_xpath("//*[@id='container']/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[4]/table/tbody/tr[2]/td[2]/ul/li")
        RAM.append(RAMtag.text)
    except:
        RAM.append('-')
        
#storage
    try:
        Storagetag = driver.find_element_by_xpath("//*[@id='container']/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[5]/table/tbody/tr/td[2]/ul/li")
        Storage.append(Storagetag.text)
    except:
        Storage.append('-')
        
#primary camera
    try:
        PrimCameratag = driver.find_element_by_xpath("//*[@id='container']/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[6]/table/tbody/tr[2]/td[2]/ul/li")
        PrimCamera.append(PrimCameratag.text.split(sep, 1)[0])
    except:
        PrimCamera.append('-')
        
#secondary camera
    try:
        SecCameratag = driver.find_element_by_xpath("//*[@id='container']/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[6]/table/tbody/tr[5]/td[2]/ul/li")
        SecCamera.append(SecCameratag.text.split(sep, 1)[1])
    except:
        SecCamera.append('-')
        
#display size
    try:
        DispSizetag = driver.find_element_by_xpath("//*[@id='container']/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[3]/table/tbody/tr[1]/td[2]/ul/li")
        DispSize.append(DispSizetag.text)
    except:
        DispSize.append('-')
        
#battery capacity
    try:
        BatteryCap_tag = driver.find_element_by_xpath("//*[@id='container']/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[9]/table/tbody/tr[9]/td[2]/ul/li")
        BatteryCap.append(BatteryCap_tag.text)
    except:
        BatteryCap.append('-')
        
#price
    try:
        pricetag = driver.find_element_by_xpath("//*[@id='container']/div/div[3]/div[1]/div[2]/div[2]/div/div[4]/div[1]/div/div[1]")
        price.append(pricetag.text)
    except:
        price.append('-')

In [12]:
Smartphone = {'Brand Name':Brand,
              'Smartphone Name':prod,
              'Price(₹)':price,
              'Colour':Colour,
              'RAM':RAM ,
              'Storage(ROM)':Storage,
              'Primary Camera':PrimCamera,
              'Secondary Camera':SecCamera,
              'Display Size':DispSize,
              'Battery Capacity':BatteryCap,
              'Product URL':produrl}

Smartphonedf  = pd.DataFrame(Smartphone)

In [13]:
Smartphonedf

,Brand Name,Smartphone Name,Price(₹),Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Product URL
0,APPLE Mobiles,"APPLE iPhone SE (Red, 64 GB)","₹30,298",Red,A13 Bionic Chip with 3rd Gen Neural Engine,64 GB,12MP Rear Camera,-,11.94 cm (4.7 inch),"Splash, Water and Dust Resistant (IP67 Rated (...",https://www.flipkart.com/apple-iphone-se-red-6...
1,APPLE Mobiles,"APPLE iPhone SE (Black, 64 GB)",-,-,-,-,-,-,-,-,https://www.flipkart.com/apple-iphone-se-black...
2,APPLE Mobiles,"APPLE iPhone SE (White, 64 GB)","₹30,298",-,-,-,-,-,-,-,https://www.flipkart.com/apple-iphone-se-white...
3,APPLE Mobiles,"APPLE iPhone SE (Black, 128 GB)","₹35,298",Black,A13 Bionic Chip with 3rd Gen Neural Engine,128 GB,12MP Rear Camera,-,11.94 cm (4.7 inch),"Splash, Water and Dust Resistant (IP67 Rated (...",https://www.flipkart.com/apple-iphone-se-black...
4,APPLE Mobiles,"APPLE iPhone SE (Red, 128 GB)","₹35,298",Red,A13 Bionic Chip with 3rd Gen Neural Engine,128 GB,12MP Rear Camera,-,11.94 cm (4.7 inch),"Splash, Water and Dust Resistant (IP67 Rated (...",https://www.flipkart.com/apple-iphone-se-red-1...
5,APPLE Mobiles,"APPLE iPhone 12 Mini (Black, 64 GB)","₹50,298",-,-,-,-,-,-,-,https://www.flipkart.com/apple-iphone-12-mini-...
6,APPLE Mobiles,"APPLE iPhone SE (White, 128 GB)","₹35,298",White,A13 Bionic Chip with 3rd Gen Neural Engine,128 GB,12MP Rear Camera,-,11.94 cm (4.7 inch),"Splash, Water and Dust Resistant (IP67 Rated (...",https://www.flipkart.com/apple-iphone-se-white...
7,APPLE Mobiles,"APPLE iPhone 11 (Green, 64 GB)",-,Green,A13 Bionic Chip,64 GB,12MP + 12MP,-,15.49 cm (6.1 inch),"Faster Face ID, Slo-mo Selfies, Water Resistan...",https://www.flipkart.com/apple-iphone-11-green...
8,APPLE Mobiles,"APPLE iPhone 12 Mini (Purple, 64 GB)","₹50,298",-,-,-,-,-,-,-,https://www.flipkart.com/apple-iphone-12-mini-...
9,APPLE Mobiles,"APPLE iPhone 11 (Green, 128 GB)",-,Green,A13 Bionic Chip,128 GB,12MP + 12MP,-,15.49 cm (6.1 inch),"Faster Face ID, Slo-mo Selfies, Water Resistan...",https://www.flipkart.com/apple-iphone-11-green...


In [14]:
Smartphonedf.to_csv('FlipkartSmartphonescrape.csv')

**5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps**

In [17]:
driver = webdriver.Chrome(r'C:\Users\Sahil\Downloads\chromedriver.exe')

In [18]:
url = "https://www.google.com/maps/"
driver.get(url)

In [19]:
item = input("Enter a location name to search:")

Enter a location name to search:New Delhi


In [20]:
search = driver.find_element_by_xpath('/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/form/div/div[3]/div/input[1]')
search.send_keys(item)
search = driver.find_element_by_xpath('/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/button')
search.click()

In [21]:
url = driver.current_url

In [22]:
url

'https://www.google.com/maps/place/New+Delhi,+Delhi/@28.5275817,77.0685573,11z/data=!3m1!4b1!4m5!3m4!1s0x390cfd5b347eb62d:0x52c2b7494e204dce!8m2!3d28.6139391!4d77.2090212'

In [23]:
import re

In [24]:
coord = re.findall(r'@(.*),',url)[0].split(",")

In [25]:
coord

['28.5275817', '77.0685573']

In [26]:
Lattitude = coord[0]

In [27]:
Longitude = coord[1]

In [31]:
print(f"Coordinates of", item, "are: ", Lattitude, ",",Longitude)

Coordinates of New Delhi are:  28.5275817 , 77.0685573


**6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.**

In [69]:
driver = webdriver.Chrome(r'C:\Users\Sahil\Downloads\chromedriver.exe')

In [42]:
url = "https://trak.in/"
driver.get(url)

In [44]:
fd = driver.find_element_by_xpath('/html/body/div[1]/header/div[2]/div/div/div/div/nav/ul/li[9]/a')
fd.click()

In [62]:
date = []
name = []
subvrt = []
indstry = []
city = []
invname = []
invtyp = []
amt = []

In [65]:
for i in range(54,57):  
    
    try:
        date_tag = driver.find_elements_by_xpath(f"//*[@id='tablepress-{i}']/tbody/tr/td[2]")
        for i in date_tag:
            date.append(i.text)
    except NoSuchElementException:
        date.append('-')
for i in range(54,57):  
 
    try:
        name_tag = driver.find_elements_by_xpath(f"//*[@id='tablepress-{i}']/tbody/tr/td[3]/a")
        for j in name_tag:
            name.append(j.text)
    except NoSuchElementException:
        name.append('-')
        
for i in range(54,57):  
    
    try:
        indstry_tag = driver.find_elements_by_xpath(f"//*[@id='tablepress-{i}']/tbody/tr/td[4]")
        for p in indstry_tag:
            indstry.append(p.text)
    except NoSuchElementException:
        indstry.append('-')    
        
for i in range(54,57):  
    
    
    try:
        subvrt_tag = driver.find_elements_by_xpath(f"//*[@id='tablepress-{i}']/tbody/tr/td[5]")
        for k in subvrt_tag:
            subvrt.append(k.text)
    except NoSuchElementException:
        subvrt.append('-')
        
for i in range(54,57):  
    
    try:
        city_tag = driver.find_elements_by_xpath(f"//*[@id='tablepress-{i}']/tbody/tr/td[6]")
        for l in city_tag:
            city.append(l.text)
    except NoSuchElementException:
        city.append('-')
        
for i in range(54,57):  
    
    try:
        invname_tag = driver.find_elements_by_xpath(f"//*[@id='tablepress-{i}']/tbody/tr/td[7]")
        for m in invname_tag:
            invname.append(m.text)
    except NoSuchElementException:
        invname.append('-')
        
for i in range(54,57):  
    
    try:
        invtyp_tag  = driver.find_elements_by_xpath(f"//*[@id='tablepress-{i}']/tbody/tr/td[8]")
        for n in invtyp_tag:
            invtyp.append(n.text)
    except NoSuchElementException:
        invtyp.append('-')
        
for i in range(54,57):  
    
    try:
        amt_tag = driver.find_elements_by_xpath(f"//*[@id='tablepress-{i}']/tbody/tr/td[9]")
        for o in amt_tag:
            amt.append(o.text)
    except NoSuchElementException:
        amt.append('-')

In [66]:
for i in range(0,len(invtyp)):
    if invtyp[i]=="":
        invtyp[i] = "-"

In [67]:
Startup = {'Date':date,'Startup Name':name,'Industry/Vertical':indstry,'Sub-Vertical':subvrt,'City/Location':city ,"Investor's Name":invname,'Investment Type':invtyp,'Amount(in USD)':amt}

Startupdf  = pd.DataFrame(Startup)

In [68]:
Startupdf

,Date,Startup Name,Industry/Vertical,Sub-Vertical,City/Location,Investor's Name,Investment Type,Amount(in USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


**7. Write a program to scrap all the available details of best gaming laptops from digit.in.**

In [70]:
driver = webdriver.Chrome(r'C:\Users\Sahil\Downloads\chromedriver.exe')

In [71]:
url = "https://www.digit.in/"
driver.get(url)

In [72]:
top10btn = driver.find_element_by_xpath('/html/body/div[1]/div/div[4]/ul/li[4]/a')
top10btn.click()

In [74]:
gaminglaptopbtn = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[3]/div[2]/ul/li[1]/a')
gaminglaptopbtn.click()

In [80]:
name = []
os = []
disp = []
mem = []
price = []
Processor = []
GraphProc= []
dim = []
wt = []
desc = []

In [81]:
name_tag  = driver.find_elements_by_xpath("//*[@id='summtable']/tbody/tr/td[1]")
for n in name_tag:
    name.append(n.text)
price_tag  = driver.find_elements_by_xpath("//*[@id='summtable']/tbody/tr/td[3]")
for p in price_tag:
    price.append(p.text)
os_tag = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[3]/td[3]")
for o in os_tag:
    os.append(o.text)
disp_tag = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[4]/td[3]")
for d in disp_tag:
    disp.append(d.text)
Processor_tag = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[5]/td[3]")
for k in Processor_tag:
    Processor.append(k.text)
GraphProc_tag = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[9]/td[3]")
for g in GraphProc_tag:
    GraphProc.append(g.text)
mem_tag = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[6]/td[3]")
for m in mem_tag:
    mem.append(g.text)
dim_tag = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[8]/td[3]")
for di in dim_tag:
    dim.append(di.text)
wt_tag = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[7]/td[3]")
for w in wt_tag:
    wt.append(w.text)
desc_tag = driver.find_elements_by_xpath("//div[@class='Section-center']//p")
for des in desc_tag:
    desc.append(des.text)

In [82]:
while('AVAILABLE' in desc):
    desc.remove("AVAILABLE")

In [83]:
while('OUT OF STOCK' in desc):
    desc.remove("OUT OF STOCK")

In [84]:
Laptops = {'Laptop Name':name,'Operating System':os,'Display': disp,'Processor':Processor,'Graphics Processor':GraphProc ,'Memory':mem,'Dimensions':dim,'Weight(kg)':wt,'Price(₹)':price,'Details':desc}

Laptops  = pd.DataFrame(Laptops)

In [85]:
Laptops

,Laptop Name,Operating System,Display,Processor,Graphics Processor,Memory,Dimensions,Weight(kg),Price(₹),Details
0,HP Victus 16,Windows 10,"16.1"" (1920 x 1200)",5th Gen AMD Ryzen 7 5800H | 3.2GHz,NVIDIA GeForce RTX 3060,NVIDIA® GeForce® GTX 1650,37 x 26.2 x 2.4,2.48,"₹ 104,990",Every list of the “best gaming laptops” needs ...
1,HP Omen 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen 7 Octa Core-5800H | 4.4 GHz,NVIDIA GeForce RTX 3060,NVIDIA® GeForce® GTX 1650,357.9 x 239.7 x 22.5,2.37,"₹ 110,790","If the Victus series does not fit your taste, ..."
2,Acer Nitro 5,Windows 10,"15.6"" (1920 x 1080)",AMD Ryzen 9 Octa Core | 2.4 GHz,NVIDIA GeForce RTX 3070,NVIDIA® GeForce® GTX 1650,363.4 x 255 x 23.9,2.4,"₹ 129,990",Possibly the best value-for-money gaming lapto...
3,MSI Stealth 15M,Windows 10,"15.6"" (1920 x 1080)",Intel Core i7 11th Gen - 11375H | NA,NVIDIA GeForce RTX 3060,NVIDIA® GeForce® GTX 1650,358.3 x 248 x 16.15,1.7,"₹ 134,990",If you’re looking for a powerful gaming laptop...
4,ASUS ROG Strix Scar 15,Windows 10,"15.6"" (2560 x 1440)",AMD Ryzen 9 Octa Core - 5900HX | 3.3 GHz,NVIDIA GeForce RTX 3080,NVIDIA® GeForce® GTX 1650,354 x 259 x 22.6,2.30,"₹ 193,990",If you want possibly the best performance poss...
5,ASUS ROG Strix Scar 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,NVIDIA® GeForce RTX™ 3070,NVIDIA® GeForce® GTX 1650,35.4 x 25.9 x 2.26,2.30,"₹ 215,990",When the ASUS ROG Strix Scar 15 ended up on ou...
6,ASUS ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Gen Ryzen 9 | 3.3 GHz,NVIDIA GeForce RTX 2060,NVIDIA® GeForce® GTX 1650,32.5 x 22.1 x 1.8,1.65,"₹ 144,990",The Asus Zephyrus G14 is a first-of-its-kind g...
7,Lenovo Legion 5i,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Gen Intel® Core™ i5-10300H | 2.50 GHz,NVIDIA® GeForce® GTX 1650 4GB,NVIDIA® GeForce® GTX 1650,363.06 x 259.61 x 23.57,2.3,"₹ 76,988",Lenovo's Legion 5i is also a suitable laptop f...
8,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,NVIDIA GeForce RTX 2070 Max-Q,NVIDIA® GeForce® GTX 1650,268.30 x 360.00 x 20.90,2.4,"₹ 185,000",The machine is powered by an Intel Core i7-108...
9,Acer Aspire 7 gaming laptop,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,NVIDIA® GeForce® GTX 1650,NVIDIA® GeForce® GTX 1650,2.29 x 36.3 x 25.4,2.15,"₹ 53,490",The Acer Aspire 7 gaming laptop is a very capa...


**8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.**

In [86]:
driver = webdriver.Chrome(r'C:\Users\Sahil\Downloads\chromedriver.exe')

In [87]:
url = "https://www.forbes.com/?sh=25ded2062254"
driver.get(url)

In [88]:
xplbtn = driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[1]/button[1]')
xplbtn.click()

In [89]:
bilbtn = driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[3]/ul/li[1]')
bilbtn.click()
allbtn =  driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a')
allbtn.click()

In [90]:
rank = []
name = []
netwrth = []
age = []
cit = []
src = []
ind = []

In [91]:
rank_tag = driver.find_elements_by_xpath("//div[@class='rank']")
name_tag = driver.find_elements_by_xpath("//div[@class='personName']//div")
netwrth_tag = driver.find_elements_by_xpath("//div[@class='netWorth']//div")
age_tag = driver.find_elements_by_xpath("//div[@class='age']//div")
cit_tag = driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
src_tag = driver.find_elements_by_xpath("//div[@class='source-column']//div")
ind_tag = driver.find_elements_by_xpath("//div[@class='category']//div")

In [92]:
for r in rank_tag:
    rank.append(r.text)
for n in name_tag:
    name.append(n.text)
for nt in netwrth_tag:
    netwrth.append(nt.text)
for a in age_tag:
    age.append(a.text)
for c in cit_tag:
    cit.append(c.text)
for s in src_tag:
    src.append(s.text)
for i in ind_tag:
    ind.append(i.text)

In [93]:
import time 


i=0

while(i<=12):
    
    nextbtn = driver.find_element_by_xpath("//div[@class='next-page']")
    nextbtn.click()
    time.sleep(3)
    
    rank_tag = driver.find_elements_by_xpath("//div[@class='rank']")
    name_tag = driver.find_elements_by_xpath("//div[@class='personName']//div")
    netwrth_tag = driver.find_elements_by_xpath("//div[@class='netWorth']//div")
    age_tag = driver.find_elements_by_xpath("//div[@class='age']//div")
    cit_tag = driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    src_tag = driver.find_elements_by_xpath("//div[@class='source-column']//div")
    ind_tag = driver.find_elements_by_xpath("//div[@class='category']//div")
    for r in rank_tag:
            rank.append(r.text)
    for n in name_tag:
            name.append(n.text)
    for nt in netwrth_tag:
            netwrth.append(nt.text)
    for a in age_tag:
            age.append(a.text)
    for c in cit_tag:
            cit.append(c.text)  
    for s in src_tag:
            src.append(s.text)
    for k in ind_tag:
            ind.append(k.text)
    
    i=i+1

In [94]:
len(rank)

2755

In [95]:
len(name)

2755

In [96]:
len(netwrth)

5510

In [97]:
while('' in netwrth):
    netwrth.remove("")

In [98]:
len(netwrth)

2755

In [99]:
len(age)

2755

In [100]:
len(cit)

2755

In [101]:
len(src)

5510

In [104]:
while('' in src):
    src.remove("")

In [105]:
len(src)

2755

In [106]:
len(ind)

2755

In [107]:
Billionaires = {'Rank':rank,'Name':name,'Net Worth':netwrth,'Age': age,'Citizenship/Country':cit,'Source':src,'Industry':ind}

BillionairesDF  = pd.DataFrame(Billionaires)

In [108]:
BillionairesDF

,Rank,Name,Net Worth,Age,Citizenship/Country,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
2750,2674.,Daniel Yong Zhang,$1 B,49,China,e-commerce,Technology
2751,2674.,Zhang Yuqiang,$1 B,65,China,Fiberglass,Manufacturing
2752,2674.,Zhao Meiguang,$1 B,58,China,gold mining,Metals & Mining
2753,2674.,Zhong Naixiong,$1 B,58,China,conglomerate,Diversified


In [109]:
file_name = 'List of Billionaires in the World 2021.xlsx'

In [112]:
import pandas as pd
BillionairesDF.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


**9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video**

In [113]:
driver = webdriver.Chrome(r'C:\Users\Sahil\Downloads\chromedriver.exe')

In [114]:
url = "https://www.youtube.com/watch?v=OUKGsb8CpF8"
driver.get(url)

In [119]:
i=0
while(i<100):
    driver.execute_script("window.scrollBy(0,500)") # scroll down to get more comments
    i+=1
while(i<402):
    driver.execute_script("window.scrollBy(0,15000)") # scroll down to get more comments
    i+=1

In [120]:
cmt = []
upvt = []
cmttime = []

In [121]:
cmt_tag = driver.find_elements_by_xpath("//*[@id='content-text']")
for i in cmt_tag:
    cmt.append(i.text.replace("\n",''))

upvt_tag = driver.find_elements_by_xpath("//*[@id='vote-count-middle']")
for j in upvt_tag:
    try: 
        upvt.append(j.text)
    except:
        upvt.append('-')
cmttime_tag = driver.find_elements_by_xpath("//*[@id='header-author']/yt-formatted-string/a")
for k in cmttime_tag:
    try: 
        cmttime.append(k.text)
    except:
        cmttime.append('-')

In [122]:
len(upvt)

500

In [123]:
len(cmt)

500

In [124]:
len(cmttime)

500

In [125]:
Youtube = {'Comment':cmt,'Commented':cmttime,'Number of upvotes':upvt}

In [126]:
YoutubeDF = pd.DataFrame(Youtube)

In [127]:
YoutubeDF

,Comment,Commented,Number of upvotes
0,Ronaldo and Messi... Never gonna be a more leg...,10 months ago,2.1K
1,The fact I dont have a preference in this Mess...,8 months ago,1.9K
2,Ronaldo's bicycle kick was so good it made Zid...,7 months ago,1.7K
3,"His move are simple, tricky, accurate and powe...",5 months ago,238
4,R-RareO-OutstandingN-Never give upA-Always pla...,6 months ago (edited),552
...,...,...,...
495,Best player in the world,5 months ago,1
496,"ADOREI, SUIIIIIIIII, PAI GRANDE, SUUUUIIIIII",3 months ago,
497,Yes Ronaldo scores only penalties & tap - ins 😀😀,4 months ago (edited),1
498,The most complete player ever.,1 year ago,5


**10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.**

In [128]:
driver = webdriver.Chrome(r'C:\Users\Sahil\Downloads\chromedriver.exe')

In [129]:
url = "https://www.hostelworld.com/pwa/wds"
driver.get(url)

In [131]:
srchbar = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div[2]/div[1]/div/div/div[1]/wds-search-form-double-l/wds-search-input/div/div/input')
srchbar.click()

In [132]:
item = 'London, England'
srchbar = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div[2]/div[1]/div/div/div[1]/wds-search-form-double-l/wds-search-input/div/div/input')
srchbar.send_keys(item)

In [134]:
suggbtn = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div[2]/div[1]/div/div/div[1]/div/wds-search-dropdown/wds-search-list/wds-search-item[1]/div[2]')
suggbtn.click()

In [135]:
gobtn =  driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div[2]/div[1]/div/div/div[1]/wds-search-form-double-l/div[2]/div[2]/wds-button/button/span')
gobtn.click()

In [136]:
name = []
dist = []
rate = []
rev = []
ovrrev = []
pvt = []
dor = []
fac = []
desc = []
url = []

In [138]:
url_tag = driver.find_elements_by_xpath("//*[@id='__layout']/div/div[1]/div[4]/div/div/div[3]/div[2]/div[1]/h2/a")
for u in url_tag:
    url.append(u.get_attribute("href"))

In [139]:
name_tag= driver.find_elements_by_xpath("//h2[@class='title title-6']//a")
for n in name_tag:
    name.append(n.text)
dist_tag= driver.find_elements_by_xpath("//a[@class='show-on-map']/span[1]")
for d in dist_tag:
    dist.append(d.text)
rate_tag= driver.find_elements_by_xpath("//div[@class='rating rating-summary-container big']/div[1]")
for r in rate_tag:
    rate.append(r.text.replace('\n0 Total Reviews',''))
rev_tag= driver.find_elements_by_xpath("//div[@class='reviews']")
for re in rev_tag:
    rev.append(re.text)
ovrrev_tag= driver.find_elements_by_xpath("//div[@class='rating rating-summary-container big']//span")
for o in ovrrev_tag:
    ovrrev.append(o.text)
pvt_tag= driver.find_elements_by_xpath("//a[@class='prices']/div[1]")
for p in pvt_tag:
    pvt.append(p.text.replace("\n",''))
dor_tag= driver.find_elements_by_xpath("//a[@class='prices']/div[2]")
for d in dor_tag:
    dor.append(d.text.replace("\n",''))
fac_tag= driver.find_elements_by_xpath("//div[@class='facilities-label facilities']")
for f in fac_tag:
    fac.append(f.text.replace("\n",' + '))

In [140]:
import time 


i=0

while(i<2):
    nextbtn = driver.find_element_by_xpath("//i[@class='core-icon icon-core-chevron-right']")
    nextbtn.click()
    time.sleep(5)
    url_tag = driver.find_elements_by_xpath("//a[@class='view-button']")
    for u in url_tag:
        url.append(u.get_attribute("href"))
    
    name_tag= driver.find_elements_by_xpath("//h2[@class='title title-6']//a")
    for n in name_tag:
        name.append(n.text)
    dist_tag= driver.find_elements_by_xpath("//a[@class='show-on-map']/span[1]")
    for d in dist_tag:
        dist.append(d.text)
    rate_tag= driver.find_elements_by_xpath("//div[@class='rating rating-summary-container big']/div[1]")
    for r in rate_tag:
        rate.append(r.text.replace('\n0 Total Reviews',''))
    rev_tag= driver.find_elements_by_xpath("//div[@class='reviews']")
    for re in rev_tag:
        rev.append(re.text)
    ovrrev_tag= driver.find_elements_by_xpath("//div[@class='rating rating-summary-container big']//span")
    for o in ovrrev_tag:
        ovrrev.append(o.text)
    pvt_tag= driver.find_elements_by_xpath("//a[@class='prices']/div[1]")
    for p in pvt_tag:
        pvt.append(p.text.replace("\n",''))
    dor_tag= driver.find_elements_by_xpath("//a[@class='prices']/div[2]")
    for d in dor_tag:
        dor.append(d.text.replace("\n",''))
    fac_tag= driver.find_elements_by_xpath("//div[@class='facilities-label facilities']")
    for f in fac_tag:
        fac.append(f.text.replace("\n",' + '))
    
    
    i=i+1

In [141]:
for u in url:
    driver.get(u)
    time.sleep(3)
    desc_tag = driver.find_element_by_xpath("//div[@class='collapsible']//div[2]")
    try:
        desc.append(desc_tag.text)
    except:
        desc.append('-')

In [144]:
Hostelworld = {'Hostel Name': name,'Distance From City Centre':dist,'Ratings':rate,'Total Reviews':rev,'Overall Reviews': ovrrev,"Privates From":pvt,'Dormitories From':dor,'Facilities':fac,'Property Description':desc}

In [149]:
print(len(name))
print(len(dist))
print(len(rate))
print(len(rev))
print(len(ovrrev))
print(len(pvt))
print(len(dor))
print(len(fac))
print(len(desc))
print(len(url))

78
78
78
78
78
76
78
78
49
49


In [151]:
Hostelworld = {'Hostel Name': name,'Distance From City Centre':dist,'Ratings':rate,'Total Reviews':rev,'Overall Reviews': ovrrev,'Dormitories From':dor,'Facilities':fac}

In [152]:
Hostelworlddf = pd.DataFrame(Hostelworld)

In [153]:
Hostelworlddf

,Hostel Name,Distance From City Centre,Ratings,Total Reviews,Overall Reviews,Dormitories From,Facilities
0,St Christopher's Village,Hostel - 1.8km from city centre,8.2,11013 Total Reviews,Fabulous,Dorms FromRs1408.57 Rs1268,Free WiFi + Follows Covid-19 sanitation guidance
1,Generator London,Hostel - 3km from city centre,7.8,6823 Total Reviews,Very Good,Dorms FromRs1742,Free WiFi + Follows Covid-19 sanitation guidance
2,Safestay London Elephant & Castle,Hostel - 1.7km from city centre,8.0,4064 Total Reviews,Fabulous,Dorms FromRs1827,Free WiFi + Follows Covid-19 sanitation guidance
3,Smart Camden Inn Hostel,Hostel - 4.4km from city centre,8.6,2757 Total Reviews,Fabulous,Dorms FromRs1638,Free WiFi + Follows Covid-19 sanitation guidance
4,Pickwick Hall,Hostel - 2.3km from city centre,9.0,2286 Total Reviews,Superb,No Dorms Available,Free Breakfast
...,...,...,...,...,...,...,...
73,The Dover,Hotel - 1.9km from city centre,No Rating\n4 Total Reviews,4 Total Reviews,No Rating,No Dorms Available,Free WiFi + Free Breakfast
74,TLK Apartments & Hotel,Hotel - 19.9km from city centre,No Rating,0 Total Reviews,No Rating,No Dorms Available,Free WiFi
75,Park Hotel,Hotel - 4.9km from city centre,No Rating,0 Total Reviews,No Rating,No Dorms Available,Free WiFi
76,Hanover Hotel,Hotel - 2.1km from city centre,No Rating\n46 Total Reviews,46 Total Reviews,No Rating,No Dorms Available,Free WiFi + Free Breakfast
